<a href="https://colab.research.google.com/github/quazirab/fine-tuning-llama-3.1-on-medical-questionnaires/blob/data_prep_for_medical_question_training/notebooks/data_prep_for_medical_question_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup
Install the following python packages

In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", wh

# Data prep

In [2]:
from datasets import load_dataset
dataset = load_dataset("lavita/medical-qa-datasets", name="medmcqa", split="train")

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [29]:
dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [3]:
df = dataset.to_pandas()
df.head(5)

,id,question,opa,opb,opc,opd,cop,choice_type,exp,subject_name,topic_name
0,e9ad821a-c438-4965-9f77-760819dfa155,Chronic urethral obstruction due to benign pri...,Hyperplasia,Hyperophy,Atrophy,Dyplasia,2,single,Chronic urethral obstruction because of urinar...,Anatomy,Urinary tract
1,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,Which vitamin is supplied from only animal sou...,Vitamin C,Vitamin B7,Vitamin B12,Vitamin D,2,single,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...,Biochemistry,Vitamins and Minerals
2,5c38bea6-787a-44a9-b2df-88f4218ab914,All of the following are surgical options for ...,Adjustable gastric banding,Biliopancreatic diversion,Duodenal Switch,Roux en Y Duodenal By pass,3,multi,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba...",Surgery,Surgical Treatment Obesity
3,cdeedb04-fbe9-432c-937c-d53ac24475de,Following endaerectomy on the right common car...,Central aery of the retina,Infraorbital aery,Lacrimal aery,Nasociliary aretry,0,multi,The central aery of the retina is a branch of ...,Ophthalmology,None
4,dc6794a3-b108-47c5-8b1b-3b4931577249,Growth hormone has its effect on growth through?,Directly,IG1-1,Thyroxine,Intranuclear receptors,1,single,"Ans. is 'b' i.e., IGI-1GH has two major functi...",Physiology,None


For this traning set, lets only use the single choice_type and create a new column with answer from the choice_type and option A, B, C and D

In [4]:
def answer(row):
  match row["cop"]:
    case 0:
      return row["opa"]
    case 1:
      return row["opb"]
    case 2:
      return row["opc"]
    case 3:
      return row["opd"]

df["answer"] = df.apply(answer, axis=1)

# pick only the required columns
df = df[["question", "answer", "exp"]]

df.head(5)

,question,answer,exp
0,Chronic urethral obstruction due to benign pri...,Atrophy,Chronic urethral obstruction because of urinar...
1,Which vitamin is supplied from only animal sou...,Vitamin B12,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...
2,All of the following are surgical options for ...,Roux en Y Duodenal By pass,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba..."
3,Following endaerectomy on the right common car...,Central aery of the retina,The central aery of the retina is a branch of ...
4,Growth hormone has its effect on growth through?,IG1-1,"Ans. is 'b' i.e., IGI-1GH has two major functi..."


In [5]:
if False: df.to_csv("medical-question-w-answer-and-explanation.csv")

## Clean up
Its observered that the explanations have some repeations of answer choice. Lets try to clean that up!

In [6]:
df

,question,answer,exp
0,Chronic urethral obstruction due to benign pri...,Atrophy,Chronic urethral obstruction because of urinar...
1,Which vitamin is supplied from only animal sou...,Vitamin B12,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...
2,All of the following are surgical options for ...,Roux en Y Duodenal By pass,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba..."
3,Following endaerectomy on the right common car...,Central aery of the retina,The central aery of the retina is a branch of ...
4,Growth hormone has its effect on growth through?,IG1-1,"Ans. is 'b' i.e., IGI-1GH has two major functi..."
...,...,...,...
182817,Most common site for extra mammary Paget&;s di...,Vulva,.It is superficial manifestation of an intradu...
182818,Inferior Rib notching is seen in all except?,Neurofibromatosis,Answer is D (Neurofibromatosis) Neurofibromato...
182819,Which is false regarding cryptococcus neoformans?,Urease negative,"Ans. is 'c' i e., Urease negative Cryptococcus..."
182820,Histopathological finding of gluten hypersensi...,Crypt hyperplasia,"Ans. is 'a' i.e., Crypt hyperplasia Histopatho..."


Observing the dataset, the majority of repetation is based like -
regex pattern - `Ans.* [^\w\s][A-Za-z][^\w\s]`
* Ans. (c)
* Ans. is 'd'
* Ans. is 'b'
* Ans. C
* Ans) C
* Ans : B
* Answer is D
* Ans is 'c'
* Answer- B.
* Ans: (b)
* Ans A
* Ans, is 'a'
* Ans is 'c' i.e.
* Ans: a
* Ans is (a)
* Ans, is 'a'
* Ans is 'c'


* ANSWER: A


Further pattern matching -

In [22]:
import re

re_subs = [
    "Ans.* [^\w\s][A-Za-z][^\w\s]", # clean up the Answers
    "ANS.* [^\w\s][A-Za-z][^\w\s]", # clean up the ANSWERS
    "^\S*\s", # clean up lines with whitespaces
    "^i\.e\.", # clean up lines starting with .i.e
    "^[^\w\s]", # clean up any symbols
    "^\s", # clean up any further whitespaces
    ]

def clean_ans(example: str | None):
  if example:
    for re_sub in re_subs:
      example = re.sub(re_sub, "", example)
  return example

print(clean_ans("Ans. (C). Adequate liquor amniiThe photograph shows maceration, a sign of intrauterine death.Conditions favoring maceration: Intact membranes; adequate liquor amnii & NO air. "))

Adequate liquor amniiThe photograph shows maceration, a sign of intrauterine death.Conditions favoring maceration: Intact membranes; adequate liquor amnii & NO air. 


In [23]:
df["exp"] = df["exp"].apply(clean_ans)
df

,question,answer,exp
0,Chronic urethral obstruction due to benign pri...,Atrophy,urethral obstruction because of urinary calcul...
1,Which vitamin is supplied from only animal sou...,Vitamin B12,Vitamin B12 Ref: Harrison's 19th ed. P 640* Vi...
2,All of the following are surgical options for ...,Roux en Y Duodenal By pass,en Y Duodenal Bypass Bariatric surgical proced...
3,Following endaerectomy on the right common car...,Central aery of the retina,central aery of the retina is a branch of the ...
4,Growth hormone has its effect on growth through?,IG1-1,has two major functions :-i) Growth of skeleta...
...,...,...,...
182817,Most common site for extra mammary Paget&;s di...,Vulva,is superficial manifestation of an intraductal...
182818,Inferior Rib notching is seen in all except?,Neurofibromatosis,Neurofibromatosis) Neurofibromatosis is associ...
182819,Which is false regarding cryptococcus neoformans?,Urease negative,"i e., Urease negative Cryptococcus neoformans ..."
182820,Histopathological finding of gluten hypersensi...,Crypt hyperplasia,hyperplasia Histopathological findings of glut...


In [25]:
i = 0

for x in df.itertuples():
  exp = x[3]
  if exp and "ans" in exp.lower():
    i += 1
    print(exp)

  if i == 20:
    break

Vitamin B12 Ref: Harrison's 19th ed. P 640* Vitamin B12 (Cobalamin) is synthesized solely by microorganisms.* In humans, the only source for humans is food of animal origin, e.g., meat, fish, and dairy products.* Vegetables, fruits, and other foods of nonanimal origin doesn't contain Vitamin B12 .* Daily requirements of vitamin Bp is about 1-3 pg. Body stores are of the order of 2-3 mg, sufficient for 3-4 years if supplies are completely cut off.
of the most impoant pharmacokinetic changes associated with aging is decreased renal elimination of drugs. After age 40, creatinine clearance decreases an average of 8 mL/min/1.73 m2/decade; however, the age-related decrease varies substantially from person to person. Serum creatinine levels often remain within normal limits despite a decrease in GFR because older adults generally have less muscle mass and are generally less physically active than younger adults and thus produce less creatinine. Maintenance of normal serum creatinine levels ca

Looking at the data, it seems like most of the dataset has been cleaned up with Answers mentioned at the starting. Now its time to save the dataset for Llama training. To save the dataset, set the `if False` to `if True`. It will save it the google drive from which it will be retrived by another notebook

In [28]:
if True: df.to_parquet("/content/drive/MyDrive/colab_drive/medical-question-w-answer-and-explanation-training-dataset.parquet")